In [ ]:
!pip install rembg opencv-python pillow numpy onnxruntime

In [ ]:
from rembg import remove
from PIL import Image, ImageFilter
import numpy as np
import cv2
from io import BytesIO
from google.colab import files

In [ ]:
uploaded = files.upload()

input_path = next(iter(uploaded))
print("File uploaded:", input_path)

In [ ]:
def smooth_edges(img_rgba, blur_radius=3):
    r, g, b, a = img_rgba.split()  
    a_blurred = a.filter(ImageFilter.GaussianBlur(radius=blur_radius))
    smooth_img = Image.merge("RGBA", (r, g, b, a_blurred))
    return smooth_img

def process_image(input_path, output_path, bg_color=(255, 0, 0), size=(413, 591)):
    input_image = Image.open(input_path).convert("RGB")
    with open(input_path, 'rb') as i:
        input_bytes = i.read()
        result_bytes = remove(input_bytes)
    img_rgba = Image.open(BytesIO(result_bytes)).convert("RGBA")
    img_smoothed = smooth_edges(img_rgba, blur_radius=2)
    red_bg = Image.new("RGBA", img_smoothed.size, bg_color + (255,))
    combined = Image.alpha_composite(red_bg, img_smoothed)
    combined = combined.filter(ImageFilter.SMOOTH)
    combined = combined.filter(ImageFilter.DETAIL)
    combined = combined.filter(ImageFilter.SHARPEN)

    final = combined.convert("RGB")
    final.save(output_path, "JPEG")
    print("✔️ Foto selesai diproses:", output_path)

In [ ]:
output_filename = "pas_foto_merah.jpg"
process_image(input_path, output_filename)

In [ ]:
files.download(output_filename)